In [37]:
def get_residues(lines):
    resi = []
    resi_xyz = []
    count = 0
    s = []
    xyz = []
    names = []
    name = ''
    for line in lines:
        toks = line.split()
        if len(toks) == 0:
            continue
        if toks[0] == 'ATOM' and toks[2] != 'H':
            if toks[5] != count:
                if s != []:
                    resi.append(s)
                    resi_xyz.append(xyz)
                    s = []
                    xyz = []
                count = toks[5]
                names.append(toks[3])
            s.append(toks[2])
            xyz.append([float(toks[6]), float(toks[7]), float(toks[8])])
            name = toks[3]
        
    if s != []:
        resi.append(s)
        resi_xyz.append(xyz)
    return [resi, resi_xyz, names]

In [38]:
def interface(first, first_xyz, second, second_xyz):
    result = []
    
    for i in range(0, len(first)):
        two = 0
        for j in range(0, len(first[i])):
            if first[i][j] != 'CA' and first[i][j] != 'CB':
                continue
            for k in range(0, len(second)):
                for l in range(0, len(second[k])):
                    if second[k][l] != first[i][j]:
                        continue
                    dist = ((first_xyz[i][j][0] - second_xyz[k][l][0])**2 + (first_xyz[i][j][1] - second_xyz[k][l][1])**2 + (first_xyz[i][j][2] - second_xyz[k][l][2])**2)**0.5
                    if first[i][j] == 'CA':
                        if dist < 12:
                            result.append([i, two])
                            two = k
                            break
                    if first[i][j] == 'CB':
                        if dist < 6:
                            result.append([i, two])
                            two = k
                            break
    return result

In [39]:
residues = {
    'ILE': 0,
    'VAL': 1,
    'LEU': 2,
    'PHE': 3,
    'CYS': 4,
    'MET': 5,
    'ALA': 6,
    'GLY': 7,
    'THR': 8,
    'SER': 9,
    'TRP': 10,
    'TYR': 11,
    'PRO': 12,
    'HIS': 13,
    'GLU': 14,
    'GLN': 15,
    'ASP': 16,
    'ASN': 17,
    'LYS': 18,
    'ARG': 19
}

In [60]:
import os
import glob
import numpy as np

np.set_printoptions(precision=3, suppress=True)

pathes = glob.glob('data/*.pdb')
C = np.zeros((20,20), dtype = np.uint32)
F = np.zeros(20)
for i in range(0, len(pathes), 2):
    file_1 = open(pathes[i])
    pdb_1 = file_1.read()
    lines_1 = pdb_1.split('\n')
    first, first_xyz, first_names = get_residues(lines_1)
    file_2 = open(pathes[i + 1])
    pdb_2 = file_2.read()
    lines_2 = pdb_2.split('\n')
    second, second_xyz, second_names = get_residues(lines_2)
    resi = interface(first, first_xyz, second, second_xyz)
    for item in resi:
        C[residues[first_names[item[0]]]][residues[second_names[item[1]]]] += 1
        C[residues[second_names[item[1]]]][residues[first_names[item[0]]]] += 1
    resi_1 = []
    resi_2 = []
    for item in resi:
        if item[0] not in resi_1:
            resi_1.append(item[0])
        if item[1] not in resi_2:
            resi_2.append(item[1])
    for item in resi_1:
        F[residues[first_names[item]]] += 1
    for item in resi_2:
        F[residues[second_names[item]]] += 1
print('C:')
print(C)
print('F:')
print(F)

C:
[[24 17 28 22  7  7 32 20 18 28  3 20 14  4 28 16 28 23 23 22]
 [17 36 41 25  9 15 47 36 27 37  9 23 27  6 42 14 17 18 34 35]
 [28 41 76 51 12 18 47 34 22 66  9 35 28 12 37 24 29 17 34 44]
 [22 25 51 30 13 12 43 21 26 34 15 25 23 11 29 18 24 17 30 24]
 [ 7  9 12 13  0  8  9  3  4 28  2  4 17  1  8  6  1  9 16 13]
 [ 7 15 18 12  8  8 16 10 13 10  3 18 13  9 19 12 12  6 16 11]
 [32 47 47 43  9 16 54 26 33 43 12 27 29  7 40 23 37 17 43 36]
 [20 36 34 21  3 10 26 40 29 52  7  8 25 14 31 28 37 19 37 25]
 [18 27 22 26  4 13 33 29 22 41  6 17 18  3 22 14 29  9 28 22]
 [28 37 66 34 28 10 43 52 41 56 13 34 33 21 52 29 49 30 31 31]
 [ 3  9  9 15  2  3 12  7  6 13  6  8 14  3  7  8 10  5 12  3]
 [20 23 35 25  4 18 27  8 17 34  8 22 21 10 21 21 28 19 30 22]
 [14 27 28 23 17 13 29 25 18 33 14 21 26  5 24 17 24 14 27 11]
 [ 4  6 12 11  1  9  7 14  3 21  3 10  5  2 14 10 14  5  4 14]
 [28 42 37 29  8 19 40 31 22 52  7 21 24 14 50 19 36 15 42 31]
 [16 14 24 18  6 12 23 28 14 29  8 21 17 10 19 16 23

In [61]:
A = 10
W = F / sum(F)
print('W:')
print(W)

tr = 0
for i in range(20):
    for j in range(i + 1):
        tr += C[i][j]
        
Q = C / tr
print('Q:')
print(Q)

W:
[0.042 0.061 0.096 0.045 0.029 0.021 0.053 0.062 0.062 0.059 0.019 0.05
 0.047 0.028 0.074 0.039 0.059 0.044 0.059 0.05 ]
Q:
[[0.005 0.004 0.006 0.005 0.002 0.002 0.007 0.004 0.004 0.006 0.001 0.004
  0.003 0.001 0.006 0.004 0.006 0.005 0.005 0.005]
 [0.004 0.008 0.009 0.005 0.002 0.003 0.01  0.008 0.006 0.008 0.002 0.005
  0.006 0.001 0.009 0.003 0.004 0.004 0.007 0.008]
 [0.006 0.009 0.017 0.011 0.003 0.004 0.01  0.007 0.005 0.015 0.002 0.008
  0.006 0.003 0.008 0.005 0.006 0.004 0.007 0.01 ]
 [0.005 0.005 0.011 0.007 0.003 0.003 0.009 0.005 0.006 0.007 0.003 0.005
  0.005 0.002 0.006 0.004 0.005 0.004 0.007 0.005]
 [0.002 0.002 0.003 0.003 0.    0.002 0.002 0.001 0.001 0.006 0.    0.001
  0.004 0.    0.002 0.001 0.    0.002 0.004 0.003]
 [0.002 0.003 0.004 0.003 0.002 0.002 0.004 0.002 0.003 0.002 0.001 0.004
  0.003 0.002 0.004 0.003 0.003 0.001 0.004 0.002]
 [0.007 0.01  0.01  0.009 0.002 0.004 0.012 0.006 0.007 0.009 0.003 0.006
  0.006 0.002 0.009 0.005 0.008 0.004 0.009 0.00

In [63]:
G = np.zeros((20,20))

for i in range(20):
    for j in range(20):
        if Q[i][j] != 0:
            G[i][j] = A * math.log(Q[i][j] / W[i] / W[j], 10)
print('G:')
print(G)

G:
[[ 4.73   1.629  1.843  4.052  1.063  2.389  4.948  2.229  1.771  3.905
  -0.799  3.159  1.947 -1.252  2.925  3.291  3.96   4.392  3.106  3.573]
 [ 1.629  3.285  1.896  3.004  0.551  4.096  5.015  3.179  1.929  3.512
   2.37   2.163  3.196 -1.094  3.083  1.108  0.19   1.725  3.2    3.986]
 [ 1.843  1.896  2.623  4.147 -0.153  2.934  3.062  0.977 -0.913  4.072
   0.416  2.033  1.401 -0.037  0.579  1.495  0.556 -0.476  1.247  3.027]
 [ 4.052  3.004  4.147  5.1    3.451  4.43   5.932  2.141  3.069  4.448
   5.891  3.828  3.803  2.842  2.777  3.503  2.991  2.78   3.96   3.651]
 [ 1.063  0.551 -0.153  3.451  0.     4.653  1.123 -4.326 -3.077  5.589
  -0.876 -2.147  4.474 -5.589 -0.832  0.715 -8.827  2.002  3.214  2.972]
 [ 2.389  4.096  2.934  4.43   4.653  5.979  4.948  2.229  3.368  2.443
   2.212  5.712  4.635  5.28   4.251  5.052  3.291  1.567  4.54   3.573]
 [ 4.948  5.015  3.062  5.932  1.123  4.948  6.19   2.338  3.373  4.737
   4.191  3.432  4.079  0.148  3.443  3.836  4.14   2.0